In [44]:
# imports
import requests
import pprint
from bs4 import BeautifulSoup

# assign the root url (without /status) to the root_url variable for ease of reference
root_url = "https://country-leaders.onrender.com/"

# assign the different endpoints
status_url = "status/"
countries_url = 'countries/'
cookie_url = 'cookie/'
leaders_url = 'leaders/'
leader_url = 'leader/'

# query the /status endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(root_url + status_url) 

# check the status_code using a condition and print appropriate messages (4 lines)
if response.status_code == 200:
    print("Success:", response.status_code)
else:
    print("Error:", response.status_code)

# Get cookies
cookie_req = requests.get(root_url + cookie_url)
cookies = cookie_req.cookies

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(root_url + countries_url, cookies=cookies) 

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json() #['ru', 'be', 'ma', 'us', 'fr']

# Initialize the dictionary to store leaders per country
leaders_per_country = {}

# display the request's status code and the countries variable
for country in countries: # ru, be, fr...
    req = requests.get(root_url + leaders_url, params={'country': country}, cookies=cookies)

     # Initialize an empty list for leaders of the current country
    leaders = []
    leader_id = ''

    for leader in req.json(): 
        # def leader_id var
        leader_id = leader['id']

        #find first par wiki for leader
        first_par = find_first_bold_paragraph(leader_id)

        leaders.append(f"{leader_id} {leader['first_name']} {leader['last_name']}")

    # Store the list of leaders in the dictionary with the country as the key
    leaders_per_country[country] = leaders

# Print the leaders per country
for country, leaders in leaders_per_country.items():
    print(f"===============  {country}  ===================")
    for leader in leaders:
        print(leader)
        # find_first_bold_paragraph(leader['id'])

# get wiki for leader
l_id = requests.get(root_url + leader_url, params={'leader_id': leader_id}, cookies=cookies)
l_id = l_id.json()
pprint.pp(l_id['wikipedia_url'])


'''=============  Find first para stuff ==================='''
def find_first_bold_paragraph(wiki_url):
    # Fetch the webpage content
    response = requests.get(wiki_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all paragraphs
    paragraphs = soup.find_all('p')

    # Iterate through paragraphs
    for paragraph in paragraphs:
        # Check if the paragraph contains any bold text
        if paragraph.find('b'):
            return {
                'paragraph_text': paragraph.text.strip(),
                'bold_text': [b.text.strip() for b in paragraph.find_all('b')]
            }

    # If no paragraph is found
    return "No paragraph found with bold text."

# Example usage
wiki_url = l_id['wikipedia_url']
result = find_first_bold_paragraph(wiki_url)

# Pretty print the result
print("Result:")
pprint.pp(result['paragraph_text'])
print()  # This will add a newline after the printed result



Success: 200


NameError: name 'leader_id' is not defined